# 前処理の手順
## このNotebookのねらい
- 前処理を行う場合は、テストデータ(検証データ)の情報を使ってはいけないことを理解する
- パイプラインの組み方を理解する


## このNotebookで重要じゃないこと
- 〇〇

In [1]:
import numpy as np

# 準備

In [24]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer = load_breast_cancer()

X_trainval, X_test, y_trainval, y_test = train_test_split(cancer.data,
                                                          cancer.target,
                                                          train_size=0.80,
                                                          random_state=42)

In [25]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, random_state=42)

# だめなコード

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1, 10, 100],
}

scaler = MinMaxScaler()
scaler.fit(X_trainval)

X_trainval_scaled = scaler.transform(X_trainval)
X_test_scaled = scaler.transform(X_test)

gs = GridSearchCV(SVC(),
                  param_grid,
                  scoring='accuracy',
                  cv=skf,
                  n_jobs=-1)

gs.fit(X_trainval_scaled, y_trainval)

print('Best Accuracy', gs.best_score_)
print('Best Parameters', gs.best_params_)

Best Accuracy 0.978021978021978
Best Parameters {'C': 1, 'gamma': 1}


## なぜ、上記のコードはダメなのか？
評価するためのデータの情報を事前に使っているから！！！

すべての訓練データ(`X_trainval`)を使ってスケーリングされた訓練データ分割(`X_trainval_scaled`)を使って交差検証を行っている。


## ダメな例の図解
『Pythonではじめる機械学習』より

![image.png (337.7 kB)](https://img.esa.io/uploads/production/attachments/6586/2019/09/14/21054/ef21198f-ebb8-4383-8a06-75f7910b9b89.png)


# 正しい手順

## 正しい手順の図解
『Pythonではじめる機械学習』より
![image.png (345.8 kB)](https://img.esa.io/uploads/production/attachments/6586/2019/09/14/21054/72fd4d47-6e7b-4128-8c58-0e1b20926d1b.png)


In [27]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1, 10, 100],
}

best_params = {}
best_mean_acc = 0

all_acc = np.array([], dtype=np.float64)
all_std = np.array([], dtype=np.float64)

for params in ParameterGrid(param_grid):
    valid_scores = []

    for train_index, valid_index in skf.split(X_trainval, y_trainval):
        X_train = X_trainval[train_index]
        X_valid = X_trainval[valid_index]

        y_train = y_trainval[train_index]
        y_valid = y_trainval[valid_index]

        scaler = MinMaxScaler()

        # 検証データの情報を使わずにスケーリングをしている！(重要)
        scaler.fit(X_train)

        X_train_scaled = scaler.transform(X_train)
        X_valid_scaled = scaler.transform(X_valid)

        clf = SVC(**params)
        clf.fit(X_train_scaled, y_train)

        acc_valid = clf.score(X_valid_scaled, y_valid)
        valid_scores.append(acc_valid)

    mean_acc = np.mean(valid_scores)
    mean_std = np.std(valid_scores)
    all_acc = np.append(all_acc, mean_acc)
    all_std = np.append(all_std, mean_std)

    if mean_acc > best_mean_acc:
        best_mean_acc = mean_acc
        best_params = params

In [28]:
print(best_mean_acc)
print(best_params)

0.9757259648563996
{'C': 1, 'gamma': 1}


In [29]:
scaler = MinMaxScaler()
scaler.fit(X_trainval)

X_trainval_scaled = scaler.transform(X_trainval)
X_test_scaled = scaler.transform(X_test)

retrained_model = SVC(**best_params)
retrained_model.fit(X_trainval_scaled, y_trainval)

retrained_model.score(X_test_scaled, y_test)

0.9736842105263158

# `Pipeline` を利用するとスッキリ書ける

In [30]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('svm', SVC()),
])

param_grid_pipe = {
    'svm__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'svm__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
}

gs_pipe = GridSearchCV(pipe,
                       param_grid_pipe,
                       cv=skf,
                       scoring='accuracy',
                       n_jobs=-1)

gs_pipe.fit(X_trainval, y_trainval)

print(gs_pipe.best_score_)
print(gs_pipe.best_params_)

0.9758241758241758
{'svm__C': 1, 'svm__gamma': 1}


In [31]:
gs_pipe.score(X_test, y_test)

0.9736842105263158

# `Pipeline` と 自前実装 の結果の一致を確かめる

In [10]:
from numpy.testing import assert_almost_equal

expected_acc = gs_pipe.cv_results_['mean_test_score']
assert_almost_equal(expected_acc, all_acc, decimal=4)

expected_std = gs_pipe.cv_results_['std_test_score']
assert_almost_equal(expected_std, all_std, decimal=4)

おわり